In [19]:
from drowsiness_detection import config
from drowsiness_detection.data import load_experiment_objects_nn
from drowsiness_detection.run_grid_search_experiment import load_experiment_data
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
import numpy as np
from sktime.transformations.panel.rocket import MiniRocketMultivariate


In [20]:
experiment_id = 115
exp_config, _, _, _ = load_experiment_objects_nn(
    experiment_id=experiment_id,
    log_dir="../logs/")
window_size = exp_config["window_in_sec"]
config.set_paths(30, window_size)

# load data
X_train, X_test, y_train, y_test, _ = load_experiment_data(
    exclude_by=exp_config["exclude_by"],
    num_targets=exp_config["num_targets"],
    seed=exp_config["seed"],
    test_size=exp_config["test_size"],
    split_by_subjects=exp_config["split_by_subjects"],
    use_dummy_data=exp_config["use_dummy_data"],
    nn_experiment=exp_config["nn_experiment"],
    feature_col_indices=exp_config["feature_col_indices"])


loading from experiment with id:  115
X_train shape: (25642, 1800, 7), y_train shape: (25642,)
X_test shape: (6494, 1800, 7), y_test shape: (6494,)


In [21]:
X_test = np.swapaxes(X_test, 1,2)
X_train = np.swapaxes(X_train, 1,2)

In [22]:
num_samples = None
X_test, y_test = X_test[:num_samples], y_test[:num_samples]
X_train, y_train = X_train[:num_samples], y_train[:num_samples]
print(X_test.shape)

(6494, 7, 1800)


In [23]:
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)
X_train_transform = minirocket_multi.transform(X_train)
print(X_train_transform.shape)

(25642, 9996)


In [ ]:
classifier = RidgeClassifier(alpha=0.001, normalize=True)
classifier.fit(X_train_transform, y_train)


/home/tim/IM/venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), RidgeClassifier())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


In [ ]:
X_test_transform = minirocket_multi.transform(X_test)


In [ ]:
y_pred = classifier.predict(X_test_transform)
print(np.mean(y_test == y_pred))